In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import RANSACRegressor
from scipy.optimize import leastsq
from scipy.signal import find_peaks

def normalize_points(points):
    centroid = np.mean(points, axis=0)
    normalized_points = points - centroid
    return normalized_points, centroid

def is_straight_line(points, threshold=1e-2):
    model_ransac = RANSACRegressor().fit(points[:, 0].reshape(-1, 1), points[:, 1])
    inlier_mask = model_ransac.inlier_mask_
    outliers = np.sum(~inlier_mask)
    return outliers / len(points) < threshold

def is_star_shape(points, min_peaks=5, prominence_threshold=0.1):
    
    points, centroid = normalize_points(points)

    
    r = np.sqrt(np.sum(points**2, axis=1))
    theta = np.arctan2(points[:, 1], points[:, 0])

    
    sorted_indices = np.argsort(theta)
    r = r[sorted_indices]
    theta = theta[sorted_indices]

    
    if theta[-1] - theta[0] < 2*np.pi:
        theta = np.concatenate([theta, [theta[0] + 2*np.pi]])
        r = np.concatenate([r, [r[0]]])

    
    theta_even = np.linspace(theta[0], theta[-1], 1000)
    r_even = np.interp(theta_even, theta, r)

    
    peaks, _ = find_peaks(r_even, prominence=prominence_threshold*np.max(r_even))

    
    if len(peaks) < min_peaks:
        return False

    
    peak_spacing = np.diff(peaks)
    if np.std(peak_spacing) / np.mean(peak_spacing) > 0.2:  
        return False

    
    valleys = (peaks + np.roll(peaks, -1)) // 2
    valley_depths = r_even[peaks[:-1]] - r_even[valleys[:-1]]
    if np.min(valley_depths) < 0.3 * np.max(r_even):  
        return False

    return True

def is_circle(points, threshold=1e-2):
    centroid = np.mean(points, axis=0)
    distances = np.linalg.norm(points - centroid, axis=1)
    return np.std(distances) / np.mean(distances) < threshold

def fit_ellipse(points):
    x = points[:, 0]
    y = points[:, 1]
    D = np.vstack([x**2, x*y, y**2, x, y, np.ones_like(x)]).T
    S = np.dot(D.T, D)
    C = np.zeros([6, 6])
    C[0, 2] = C[2, 0] = 2
    C[1, 1] = -1
    try:
        _, _, V = np.linalg.svd(np.dot(np.linalg.inv(S), C))
        a = V[0, :]
        return a
    except np.linalg.LinAlgError:
        return None

def is_ellipse(points, threshold=1e-2):
    a = fit_ellipse(points)
    distances = np.sqrt(a[0]*points[:,0]**2 + a[1]*points[:,0]*points[:,1] + a[2]*points[:,1]**2 + a[3]*points[:,0] + a[4]*points[:,1] + a[5])
    return np.mean(distances) < threshold

def is_rectangle(points, threshold=1e-2):
    if len(points) != 4:
        return False
    angles = []
    for i in range(4):
        p1, p2, p3 = points[i], points[(i+1)%4], points[(i+2)%4]
        vec1 = p1 - p2
        vec2 = p3 - p2
        angle = np.arccos(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))
        angles.append(np.degrees(angle))
    return all(85 <= angle <= 95 for angle in angles)

def is_regular_polygon(points, threshold=1e-2):
    n = len(points)
    centroid = np.mean(points, axis=0)
    distances = np.linalg.norm(points - centroid, axis=1)
    angles = []
    for i in range(n):
        p1, p2 = points[i], points[(i+1)%n]
        vec1 = p1 - centroid
        vec2 = p2 - centroid
        angle = np.arccos(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))
        angles.append(np.degrees(angle))
    return np.std(distances) / np.mean(distances) < threshold and np.std(angles) < threshold

def is_rounded_rectangle(points, threshold=1e-2):
    if len(points) < 8:  
        return False

   
    centroid = np.mean(points, axis=0)
    angles = np.arctan2(points[:,1] - centroid[1], points[:,0] - centroid[0])
    sorted_points = points[np.argsort(angles)]

    
    corners = []
    for i in range(0, len(sorted_points), len(sorted_points)//4):
        corner = sorted_points[i:i+len(sorted_points)//4]
        corners.append(corner)

    if len(corners) != 4:
        return False

    
    for corner in corners:
        if not is_circular_arc(corner, threshold):
            return False

  
    for i in range(4):
        side = np.vstack((corners[i][-1], corners[(i+1)%4][0]))
        if not is_straight_line(side, threshold):
            return False

    return True

def is_circular_arc(points, threshold=1e-2):
    if len(points) < 3:
        return False

   
    x, y = points[:,0], points[:,1]
    def calc_R(xc, yc):
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f(c):
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = np.mean(points, axis=0)
    center, _ = leastsq(f, center_estimate)

   
    radii = calc_R(*center)
    return np.std(radii) / np.mean(radii) < threshold

def classify_shape(points):
    points, _ = normalize_points(points)
    if is_straight_line(points):
        return "Straight Line"
    elif is_circle(points):
        return "Circle"
    elif is_ellipse(points):
        return "Ellipse"
    elif is_rectangle(points):
        return "Rectangle"
    elif is_rounded_rectangle(points):
        return "Rounded Rectangle"
    elif is_regular_polygon(points):
        return "Regular Polygon"
    elif is_star_shape(points):
        return "Star"
    else:
        return "Irregular Shape"

def plot_shapes(shapes):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    colours = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for i, (points, shape_type) in enumerate(shapes):
        c = colours[i % len(colours)]
        ax.plot(points[:, 0], points[:, 1], c=c, linewidth=2, label=shape_type)
    ax.set_aspect('equal')
    ax.legend()
    plt.show()

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def classify_and_plot(csv_path):
    paths_XYs = read_csv(csv_path)
    shapes = []

    for path in paths_XYs:
        for points in path:
            shape_type = classify_shape(points)
            shapes.append((points, shape_type))

    plot_shapes(shapes)


csv_file_path = 'problems/isolated.csv'
classify_and_plot(csv_file_path)
